# Script for a period

In [5]:
import requests
import json
import pandas as pd
from datetime import date
import openpyxl
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [6]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [4]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text }")

Data retrieved successfully


In [121]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)


# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [ ]:
#unique_change_times = df['changeTime'].unique()
#print(unique_change_times)

['2025-02-25T09:56:44.373405+03:00' '2025-02-25T09:56:36.977327+03:00'
 '2025-02-10T15:50:50.654674+03:00' '2025-02-10T15:50:46.711076+03:00'
 '2024-12-28T14:56:51.438476+03:00' '2025-02-10T15:50:45.190525+03:00'
 '2025-02-06T14:45:40.972398+03:00' '2024-12-28T14:59:03.412003+03:00'
 '2025-02-10T15:50:43.275656+03:00' '2025-02-25T10:07:05.519407+03:00'
 '2025-01-06T19:56:55.030411+03:00' '2025-01-31T16:14:36.03518+03:00'
 '2025-02-26T15:24:26.898332+03:00' '2025-01-24T17:28:07.565413+03:00'
 '2025-01-24T17:27:45.032317+03:00' '2025-03-03T10:51:20.471041+03:00'
 '2023-06-09T17:13:45.296789+03:00' '2023-07-17T17:56:44.162491+03:00'
 '2023-09-11T19:47:52.142401+03:00' '2023-09-13T19:02:34.535107+03:00'
 '2023-09-11T17:14:54.50553+03:00' '2023-09-13T14:53:09.428335+03:00'
 '2023-12-11T23:45:14.286592+03:00' '2023-12-11T22:31:06.941346+03:00'
 '2024-07-15T12:16:26.499587+03:00' '2024-04-11T10:53:48.400039+03:00'
 '2024-05-13T10:13:17.846636+03:00' '2024-05-20T16:18:30.678177+03:00'
 '2024-0

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [122]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# List to store all campaign data
all_campaign_data_guten = []
all_campaign_data_giper = []
all_campaign_data_kitchen = []


# Iterate over each chunk and send a POST request
for idx, (chunk_guten, chunk_giper, chunk_kitchen) in enumerate(zip(campaign_chunks_guten, campaign_chunks_giper, campaign_chunks_kitchen)):
    
    # Send the POST request
    response_guten = requests.post(url, params=query_params, json=chunk_guten, headers=headers_guten)
    time.sleep(1)
    response_giper = requests.post(url, params=query_params, json=chunk_giper, headers=headers_giper)
    time.sleep(1)
    response_kitchen = requests.post(url, params=query_params, json=chunk_kitchen, headers=headers_kitchen)
    # Add a delay to avoid hitting API rate limits
    time.sleep(1)
    
    # Check the response status
    if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200:
        # Parse the JSON response
        data_guten = response_guten.json()
        data_giper = response_giper.json()
        data_kitchen = response_kitchen.json()
        all_campaign_data_guten.extend(data_guten)
        all_campaign_data_giper.extend(data_giper)
        all_campaign_data_kitchen.extend(data_kitchen)
        print("Data retrieved successfully")
        #print(f"Response for Chunk {idx + 1}: {data_guten}")
    else:
        print(f"Error for Chunk {idx + 1}: {response_guten.status_code,response_giper.status_code,response_kitchen.status_code}, {response_guten.text,response_giper.text,response_kitchen.text}")
        
        
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully
Combined Campaign Data


In [123]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2024-12-23,2025-01-07,Metz_9 SKU_23.12.2024,22255058,Кампания на паузе,Аукцион,WB-GutenTech,Wildberries
1,2100-01-01,2024-12-19,2025-01-09,Oral-B iO_Ultimate 1 шт от 19.12.2024,22192773,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2024-12-18,2024-12-18,Пылесос Zelmer ZVC9130M от 18.12.2024,22165389,Кампания на паузе,Аукцион,WB-GutenTech,Wildberries
3,2100-01-01,2024-12-10,2025-03-06,AND_Массажер MH-102_10.12.2024_ВЧ,21965547,Идут показы,Аукцион,WB-GutenTech,Wildberries
4,2100-01-01,2024-11-28,2025-01-09,Oral-b_Насадка 1шт. 3D White_28.11.2024,21698516,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
86,2024-07-23,2024-06-05,2024-06-05,Braun Триммеры,17748698,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
87,2024-07-23,2024-06-05,2024-06-05,Braun Эпиляторы,17741906,Кампания завершена,Автоматическая кампания,WB-ГиперМаркет,Wildberries
88,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,21008536,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries
89,2100-01-01,2024-10-30,2025-01-09,Миксер планет_30.10.2024,21008249,Идут показы,Автоматическая кампания,WB-KitchenAid,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [146]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2024-11-01",  # Replace with your start date
    "end": "2024-11-30"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [147]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_guten, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_guten.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


### Giper

In [148]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_giper, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_giper.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_giper_df = pd.json_normalize(all_campaign_data_giper)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_giper, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


### Kitchen

In [149]:
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Iterate over each chunk and send a POST request
for idx, chunk in enumerate(campaign_chunks):
    #print(f"Processing Chunk {idx + 1}: {chunk}")
    
    # Construct the payload for the current chunk
    payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]
    
    # Send the POST request
    response = requests.post(url, headers=headers_kitchen, json=payload)
    
    # Add a delay to avoid hitting API rate limits
    time.sleep(65)
    
    # Check the response status
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        all_campaign_data_kitchen.extend(data)
        #print(f"Response for Chunk {idx + 1}: {json.dumps(data, indent=4)}")
        print("Data retrived successfully")
    else:
        print(f"Error for Chunk {idx + 1}: {response.status_code}, {response.text}")

# Combine all campaign data into a DataFrame
campaign_kitchen_df = pd.json_normalize(all_campaign_data_kitchen)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_kitchen, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrived successfully
Combined Campaign Data


In [150]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,63865,3442,5.39,3.55,12234.72,79,1,0.03,1,17631,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 161420...",20487381,2024-11-01,2024-11-30
1,53421,2157,4.04,2.56,5522.30,143,36,1.67,38,164049,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 209076...",18870619,2024-11-01,2024-11-30
2,25963,1536,5.92,5.08,7803.51,265,35,2.28,52,238411,"[{'date': '2024-11-22T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 159488...",21578389,2024-11-22,2024-11-30
3,306835,16888,5.50,0.78,13145.95,937,59,0.35,61,314380,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 252275...",19297608,2024-11-01,2024-11-30
4,88257,6037,6.84,1.37,8242.88,601,206,3.41,268,149570,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 224944...",16530753,2024-11-01,2024-11-30
5,11358,537,4.73,1.07,575.66,24,7,1.30,7,28668,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 643312...",20139198,2024-11-01,2024-11-30
6,34340,1986,5.78,2.77,5504.90,133,10,0.50,10,42574,"[{'date': '2024-11-07T03:00:00+03:00', 'views'...",NaN,21204098,2024-11-07,2024-11-30
7,9629,179,1.86,11.97,2142.07,7,0,0.00,0,0,"[{'date': '2024-11-07T03:00:00+03:00', 'views'...",NaN,21204193,2024-11-07,2024-11-30
8,93284,3261,3.50,2.57,8369.55,221,40,1.23,41,71715,"[{'date': '2024-11-19T03:00:00+03:00', 'views'...","[{'date': '2024-11-30T00:00:00Z', 'nm': 137609...",21497844,2024-11-19,2024-11-30
9,254299,13458,5.29,0.78,10510.87,739,62,0.46,66,341610,"[{'date': '2024-11-01T03:00:00+03:00', 'views'...","[{'date': '2024-11-13T00:00:00Z', 'nm': 263764...",19297685,2024-11-01,2024-11-30


In [151]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2024-11-01 03:00:00,161420838,Слайсер для нарезки электрический SKS 100 TWIN,297,5,1.68,13.36,66.79,0,0,0.0,0,0,20487381
1,2024-11-01 03:00:00,161420838,Слайсер для нарезки электрический SKS 100 TWIN,2356,106,4.50,5.16,547.48,0,0,0.0,0,0,20487381
2,2024-11-01 03:00:00,161420838,Слайсер для нарезки электрический SKS 100 TWIN,845,36,4.26,5.59,201.37,0,0,0.0,0,0,20487381
3,2024-11-02 03:00:00,161420838,Слайсер для нарезки электрический SKS 100 TWIN,221,2,0.90,24.24,48.48,0,0,0.0,0,0,20487381
4,2024-11-02 03:00:00,161420838,Слайсер для нарезки электрический SKS 100 TWIN,1620,64,3.95,5.03,322.05,0,0,0.0,0,0,20487381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10812,2024-11-30 03:00:00,236554007,Эпилятор женский электрический Silk-epil 9-011...,24,0,0.00,0.00,5.52,0,0,0.0,0,0,21204285
10813,2024-11-30 03:00:00,236554007,Эпилятор женский электрический Silk-epil 9-011...,150,7,4.67,4.93,34.50,0,0,0.0,0,0,21204285
10814,2024-11-30 03:00:00,238818868,Эпилятор женский электрический Silk-epil 9-030,7,1,14.29,1.61,1.61,0,0,0.0,0,0,21204285
10815,2024-11-30 03:00:00,236554007,Эпилятор женский электрический Silk-epil 9-011...,118,5,4.24,5.43,27.14,0,0,0.0,0,0,21204285


In [152]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25914858,2024-11-01 03:00:00,Яйцеварка электрическая E 7,323,6,1.216667,4.926667,38.59,0,0,0.000000,0,0,18870619,WB-GutenTech,Wildberries
1,25963255,2024-11-01 03:00:00,Кухонные весы L 15,166,9,3.500000,1.916667,19.04,0,2,37.500000,2,7054,18870701,WB-GutenTech,Wildberries
2,26085467,2024-11-01 03:00:00,Вакууматор для продуктов с запайкой VC 10,291,15,5.700000,1.170000,29.07,0,0,0.000000,0,0,18870595,WB-GutenTech,Wildberries
3,46292575,2024-11-01 03:00:00,Яйцеварка электрическая автоматическая E 9,602,25,4.666667,2.556667,65.66,0,1,11.110000,1,4957,18870619,WB-GutenTech,Wildberries
4,64331255,2024-11-01 03:00:00,Электрическая зубная щетка взрослая Pro 3 3500...,667,33,4.246667,1.260000,30.21,0,2,4.316667,2,7826,20139198,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,243235996,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",145,5,4.660000,3.456667,16.46,0,0,0.000000,0,0,21008536,WB-KitchenAid,Wildberries
3796,243235997,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",1,0,0.000000,0.000000,0.02,0,0,0.000000,0,0,21008536,WB-KitchenAid,Wildberries
3797,243235999,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",245,16,6.090000,1.873333,26.69,0,0,0.000000,0,0,21008536,WB-KitchenAid,Wildberries
3798,245313750,2024-11-30 03:00:00,Тостер на 4 ломтика 5KMT4109E,45,1,1.450000,1.613333,9.68,0,0,0.000000,0,0,21006669,WB-KitchenAid,Wildberries


In [153]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25914858,2024-11-01 03:00:00,Яйцеварка электрическая E 7,323,6,38.59,0,0,0,0,18870619,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Яйцеварки_3 SKU_25.07.24,Идут показы,Автоматическая кампания
1,25963255,2024-11-01 03:00:00,Кухонные весы L 15,166,9,19.04,0,2,2,7054,18870701,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Весы кух_2 SKU_25.07.24,Кампания на паузе,Автоматическая кампания
2,26085467,2024-11-01 03:00:00,Вакууматор для продуктов с запайкой VC 10,291,15,29.07,0,0,0,0,18870595,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Вакуумные упаковщики_2 SKU_25.07.24,Идут показы,Автоматическая кампания
3,46292575,2024-11-01 03:00:00,Яйцеварка электрическая автоматическая E 9,602,25,65.66,0,1,1,4957,18870619,WB-GutenTech,Wildberries,2100-01-01,2024-07-25,2025-01-09,CASO_Яйцеварки_3 SKU_25.07.24,Идут показы,Автоматическая кампания
4,64331255,2024-11-01 03:00:00,Электрическая зубная щетка взрослая Pro 3 3500...,667,33,30.21,0,2,2,7826,20139198,WB-GutenTech,Wildberries,2100-01-01,2024-09-24,2024-09-24,ORAL-B_Щетки 3 SKU_24.09.2024,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3795,243235996,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",145,5,16.46,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
3796,243235997,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",1,0,0.02,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
3797,243235999,2024-11-30 03:00:00,"Чайник электрический 1,7 л с регулировкой темп...",245,16,26.69,0,0,0,0,21008536,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Чайники_30.10.2024,Идут показы,Автоматическая кампания
3798,245313750,2024-11-30 03:00:00,Тостер на 4 ломтика 5KMT4109E,45,1,9.68,0,0,0,0,21006669,WB-KitchenAid,Wildberries,2100-01-01,2024-10-30,2025-01-09,Тостеры_30.10.2024,Идут показы,Автоматическая кампания


## Insert the data

In [154]:
password = os.getenv('ClickHouse')

In [155]:
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [156]:
# Define the table name
table_name = 'campaign_data_wb'

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_final.to_numpy()]

# Define the column names
column_names = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks', 'sum_price',
    'advertId', 'Project', 'Marketplace', 'endTime', 'createTime', "startTime", 'name_campaign', 'status', 'type'
]

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=column_names)

print("Data inserted successfully!")

Data inserted successfully!
